In [1]:
# Copyright 2020 Bimghi Choi. All Rights Reserved.
# 예측 + 투자전략 시스템

# _*_ coding: utf-8 _*_

from tensorflow import keras
import tensorflow as tf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]='0, 1'

In [2]:
import util
import preprocess as prepro
import models
import learn
from learn import GenerateResult
import math
import datetime

In [3]:
#tf.keras.backend.set_floatx('float64')

In [4]:
import gc
gc.collect()

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')    
#    tf.config.experimental.set_visible_devices(gpus[1], 'GPU')    
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])
  except RuntimeError as e:
    # 프로그램 시작시에 가장 장치가 설정되어야만 합니다
    print(e)

In [5]:
file_name = '~/Data/kospi200f_809_0515.csv'
item_name = 'kospi200f_reinfo_809'
train_start = '2000-01-31'
train_end = '2019-01-02'
test_start = '2019-01-02'
test_end = '2020-05-15'

remove_columns = ['date', '종가', '시가', '고가', '저가']
target_column = '종가'
input_columns = []
target_type = 'rate'

model_name = 'ddaeryuble5-2'
channel = False

trans_day = 1

target_alpha = 100
future_day = 2
train_end_back = -1
n_timestep = 30
time_interval = 1
input_size = 809
n_unit = 1024
batch_size = 10
learning_rate = 0.0005
epochs = 2000

alpha = 0.05
beta = 0.01

comment = "2일 에측을 위해 4일 예측이용, 모델1은 compile, fit방식으로"

checkpoint_path = model_name + "/pred"+str(future_day)+"_trans"+str(trans_day)+".ckpt"

In [6]:
dataframe = util.read_datafile(file_name)
df = dataframe.copy()
#df = prepro.target_conversion(df, target_column, future_day, type=target_type)
for i in range(len(df[target_column]) -future_day):
    df.loc[i, target_column] = ((df.loc[i + future_day, target_column] - df.loc[i, target_column]) 
                                / df.loc[i, target_column]) * target_alpha 

In [7]:
df.head(20)

,date,시가지수(포인트),고가지수(포인트),저가지수(포인트),종가지수(포인트),수익률(%),수익률 (1주)(%),수익률 (1개월)(%),수익률 (3개월)(%),수익률 (6개월)(%),...,"주요상품선물_천연가스(선물, NYMEX)($/mmBtu)",주요상품선물_금(선물)($/ounce),주요상품선물_은(선물)($/ounce),주요상품선물_알루미늄(선물)($/ton),주요상품선물_옥수수(최근월물)(￠/bu),대두박(￠/bu),종가,시가,고가,저가
0,2000-01-31,-0.218225,-0.493106,-0.400439,-0.223565,0.212872,1.352924,-0.851790,-0.382658,-0.162615,...,1.653438,-0.853010,0.974513,0.827681,0.373581,0.614542,-0.961538,116.85,120.10,116.65
1,2000-02-01,-0.213200,-0.230417,-0.373954,-0.595373,-0.407635,1.808646,-1.074203,-1.707405,-0.490731,...,1.687061,-0.039689,0.193571,1.002058,0.594935,0.706918,3.381014,120.85,121.70,115.35
2,2000-02-02,-0.208520,-0.448886,-0.367787,-0.200398,0.696677,2.107257,-0.769928,-1.558400,0.074389,...,1.809524,0.800355,0.031266,0.069626,0.430159,0.725768,3.292528,116.55,119.70,115.95
3,2000-02-03,-0.204125,-0.216603,-0.062403,-0.021445,0.366314,1.488978,-0.625155,-1.268930,-0.079967,...,1.213642,1.515992,0.623370,0.378121,0.084769,0.352875,1.677149,119.35,120.25,116.80
4,2000-02-07,-0.200000,0.378008,0.426748,0.592789,0.949260,1.101574,1.281908,-1.288431,1.273730,...,0.689954,3.856851,0.568635,-0.322023,0.518339,0.702038,0.653862,120.40,123.45,119.75
5,2000-02-08,-0.196110,0.353748,0.677387,0.254527,-0.366454,1.138729,0.998930,-1.329397,0.942469,...,0.332052,2.657684,1.222974,-0.720402,0.474125,0.440887,1.319588,122.90,123.60,120.95
6,2000-02-09,-0.192447,0.306497,0.819824,0.658684,0.641075,1.076862,1.233568,-1.637689,0.994675,...,0.557674,3.026664,2.114708,-0.407511,0.622228,0.721716,-1.867641,122.40,124.10,121.85
7,2000-02-10,0.330923,0.617187,0.552762,0.366383,-0.311193,0.651653,0.275024,-1.607973,0.269679,...,0.812787,3.316674,1.871659,-0.157431,0.985272,1.379493,-5.575906,120.85,125.05,120.40
8,2000-02-11,0.977388,0.576306,0.048738,-0.013448,-0.428351,0.326747,0.131530,-1.828185,0.546650,...,0.675096,2.384344,0.988933,-0.154629,0.473525,0.907189,-8.274721,123.60,124.75,119.85
9,2000-02-14,-0.649462,-0.806483,-0.954529,-1.222378,-1.533955,-1.217328,-0.075927,-2.486079,0.140712,...,0.504701,1.935983,0.085505,-0.246361,0.529501,0.923675,-1.422414,118.85,119.00,115.25


In [8]:
train_start_index = max(df.loc[df['date']<=train_start].index) + time_interval*(n_timestep-1) - 1
train_end_index = max(df.loc[df['date']<=train_end].index)
base_prices = tf.reduce_mean(df.loc[train_start_index:train_end_index+1, '시가'])  

@tf.function
def loss_fn_model1_1(targets, preds):
 
    loss0 = tf.keras.losses.MSE(targets, preds)
    
    preds = tf.reshape(preds[:, n_timestep-1, :], [-1])
    targets = tf.reshape(targets[:, n_timestep-1, :], [-1])
    
   
    if alpha != 0:
        # add RRL cost - maximize downside sharp ratio

        # 1 if (pred - base) * (target - base) > 0, -1 otherwise
        F = tf.math.sign(targets*preds)
        F = tf.reshape(F, [-1])

        # calc returns from each step in batches
        R = tf.math.divide(tf.math.multiply(tf.math.abs(targets), (F - 0.00003)), base_prices)
        R = tf.reshape(R, [-1])

        # calc downside sharp ratio

        # downside returns
        DR = tf.minimum(0.0, R)
        DR = tf.reshape(DR, [-1])

        # calc. downside sharp ratio
        #s = []
        #for i in range(batch_size):
        #   std =  tf.keras.backend.std(DR[i, :, 0])
        #   s.append(tf.reduce_mean(R[i, :, 0])/tf.maximum(0.01, std))

        # calc. downside sharp ratio
        loss1 = tf.reduce_mean(R) / (tf.keras.backend.std(DR) + 0.001)
    else:
        loss1 = 0

    """
    # average profits, loss
    avg_plusR = [0.0]
    avg_minusR = [0.0]

    global num_of_profits
    global num_of_losses

    num_of_profits = 0
    num_of_losses = 0

    for i in range(batch_size):
        res = tf.cond(R[i, num_steps - 1, 0] > 0, lambda: return_one(), lambda: return_zero())
        if res == 1:
            avg_plusR.append(R[i, num_steps - 1, 0])
        else:
            avg_minusR.append(R[i, num_steps - 1, 0])
    avg_profit = tf.reduce_mean(avg_plusR) 
    avg_loss = tf.reduce_mean(avg_minusR) 
    """

    if beta != 0:
        #compute maximum drawdown

        #accm_profit = [0.0]
        #for i in range(batch_size):
        #    for j in range(num_steps):
        #        r = tf.cond((predict_prices[i, num_steps-1, 0] - base_prices[i, num_steps-1, 0]) *
        #                   (target_prices[i, num_steps-1, 0] - base_prices[i, num_steps-1, 0]) > 0,
        #                   lambda: return_one(),
        #                   lambda: return_zero())
        #        if r == 1: accm_profit.append(accm_profit[i*num_steps + j] + tf.abs(target_prices[i, j, 0] - base_prices[i, j, 0]))
        #        else:      accm_profit.append(accm_profit[i*num_steps + j] - tf.abs(target_prices[i, j, 0] - base_prices[i, j, 0]))

        accm_profit = [0.0 for i in range(batch_size)]
        for i in range(batch_size):
            if i == 0:
                accm_profit[0] = tf.sign(preds * targets) * tf.math.abs(targets)
            else:
                accm_profit[i] = accm_profit[i-1] + tf.sign(preds * targets) * tf.math.abs(targets)
        loss2 = (tf.reduce_max(accm_profit) - tf.reduce_min(accm_profit))/batch_size
    else:
        loss2 = 0

    return loss0 + beta*loss2 - alpha*loss1

In [9]:
def make_remaining_days(n_batches, n_steps):
    remaining_days = []
    for i in range(n_batches):
        temp = []    
        for j in range(n_steps):
            temp.append((n_batches - i) / n_batches)
        remaining_days.append(np.array(temp))
    return np.array(remaining_days)

In [10]:
def reduce_mean(returns):
    sum = 0
    for i in range(batch_size):
        sum += returns[i]
    return sum / batch_size

In [11]:
@tf.function
def loss_fn_model1(train_y, preds):
    
    #preds = tf.cast(preds, dtype=tf.float64)
    #train_y = tf.cast(train_y, dtype=tf.float64)
    rates = train_y / 100
    updown = tf.math.sign(train_y)
    profits = 1 + rates*tf.math.sign(preds)
    
    loss1 = keras.losses.MSE(preds, train_y)
    
    batches = tf.cast(tf.shape(train_y)[0], dtype=tf.int32)

    return_plus = 0.000000000
    return_minus = 0.000000000
    for i in range(batches):
        if profits[i, -1, 0] - 1 > 0: return_plus += profits[i, -1, 0]-1      
        else: return_minus += 1-profits[i, -1, 0]    

    loss2 = (return_plus - return_minus)/tf.cast(batches, dtype=tf.float32)
    
    return loss1 - loss2


In [12]:
#@tf.function
def loss_fn_model2(m1, m2, train_x, train_y):
    
    rates = train_y / 100
    updown = tf.math.sign(train_y)
    preds = tf.cast(tf.math.sign(m1(train_x)), dtype=tf.float64)
    profits = tf.cast(1 + tf.convert_to_tensor(rates, dtype=tf.float64)*preds, dtype=tf.float64)
    
    batches = train_y.shape[0]
    steps = train_y.shape[1]     

    n = - int(future_day/time_interval) # calc the number of unkown steps    
    
    # input = [preds, profits, the rate of remaining days]
    #train_x_m2 = np.zeros((batches, steps, 3))
    #train_x_m2[:, :, 0] = np.sign(preds[:, :, 0])
    #train_x_m2[:, :, 1] = profits[:, :, 0]
    #train_x_m2[:, :, 2] = make_remaining_days(batches, steps)
    
    # unknown future profits are relaced by average profits
    #for i in range(len(train_x_m2)):
    #    train_x_m2[i, n:, 1] = tf.math.reduce_mean(train_x_m2[i, :n, 1])

    train_x_m2 = tf.concat([preds, rates], 2)
    #train_x_m2 = tf.concat([train_x_m2, make_remaining_days(batches, steps).reshape((batches, steps, 1))], 2)    

    # model2의 output이 목표일까지의 남은 일수의 기하평균 수익률이 되도록 학습
    targets = np.zeros(len(profits))
    
    targets[:-1] = tf.reshape(profits[1:, -1, 0], [-1]) 
    
    loss = keras.losses.MSE(tf.reshape(m2(train_x_m2)[:, -1, 0], [-1]), targets)

  
    return loss


In [13]:
#@tf.function
def test(m1, m2, test_x, test_y):    
    rates = test_y / 100
    n = - int(future_day/time_interval)
    for i in range(len(rates)):
        rates[i, n:, 0] = 0#tf.math.reduce_mean(rates[i, :n, 0])
    preds = tf.cast(tf.math.sign(m1.predict(test_x)), dtype=tf.float64)
    #rates = tf.cast(tf.random.normal((), mean=test_y, stddev=1) / 100, dtype=tf.float64)
    profits = 1 + tf.convert_to_tensor(rates, dtype=tf.float64)*preds
    
    batches = test_y.shape[0]
    steps = test_y.shape[1]     

    test_x_m2 = np.concatenate((preds, rates), axis=2).reshape((-1, n_timestep, 2))
    
    avg_return = tf.math.reduce_prod(tf.reshape(m2(test_x_m2)[:, -1, 0], [-1]))**(1/batches)
    
    for k in range(10000):
        new_test_x_m2 = test_x_m2.copy()
        for i in range(batches):
            new_test_x_m2[i, :, 0] = max(-1, min(1, tf.cast(tf.random.normal((), mean=test_x_m2[i, -1, 0], stddev=1), dtype=tf.float64)))

        new_avg_return = tf.math.reduce_prod(tf.reshape(m2(new_test_x_m2)[:, -1, 0], [-1]))**(1/batches)
        if new_avg_return > avg_return:
            test_x_m2 = new_test_x_m2.copy()
            avg_return = new_avg_return
        
    return np.array(test_x_m2[:, -1, 0]).reshape(-1)

In [14]:
#@tf.function
def gradient1(model1, input_data, output_data):
    with tf.GradientTape() as tape:
        loss = loss_fn_model1(model1, input_data, output_data)
    return tape.gradient(loss, model1.trainable_variables)
#@tf.function
def gradient2(model1, model2, input_data, output_data):
    with tf.GradientTape() as tape:
        loss = loss_fn_model2(model1, model2, input_data, output_data)
    return tape.gradient(loss, model2.trainable_variables)

In [15]:

#strategy = tf.distribute.MirroredStrategy()
#with strategy.scope():
model1 = models.LSTM(n_timestep,input_size,n_unit,regularizers_alpha=0.01,drop_rate=0.5)
model2 = models.LSTM(n_timestep,2,10,regularizers_alpha=0.01,drop_rate=0.5)

    #cp_callback = tf.keras.callbacks.ModelCheckpoint(
    #    checkpoint_path, verbose=1, save_weights_only=True,
        # 다섯 번째 에포크마다 가중치를 저장합니다
    #    save_freq=5)

model1.compile(optimizer='adam',
              loss=loss_fn_model1_1)
                  #callbacks=[cp-callback]
              #metrics=['accuracy'])
model1.save_weights(checkpoint_path)            

#    model2 = models.LSTM(n_timestep,input_size,n_unit,regularizers_alpha=0.01,drop_rate=0.5)
#    model2.compile(optimizer='adam',
#                  loss='mse')
                  #callbacks=[cp-callback]
              #metrics=['accuracy'])            

#modle_name = model_name + "tanh"            
#model2.save_weights("modle2_"+ checkpoint_path)

In [ ]:
current_train_start = df.loc[prepro.date_to_index(df, train_start), 'date']
current_train_end = df.loc[prepro.date_to_index(df, train_end), 'date']
current_test_start = df.loc[prepro.date_to_index(df, test_start), 'date']
current_test_end = df.loc[prepro.date_to_index(df, test_start) + trans_day - 1, 'date']


#  각 transfer 구간의 예측값들을 합치기 위하여
test_prediction1 = []
test_prediction2 = []
test_target = []

while True:

    gc.collect()
    
    train_data, test_data = prepro.get_train_test_data(df, target_column, remove_columns, 
                                                           current_train_start, current_train_end,
                                                           current_test_start, current_test_end,
                                                           future_day, n_timestep, time_interval)
    
    # input_size, columns reset
    input_size = len(df.columns) - len(remove_columns)
    input_columns = df.columns.copy()

    train_x, train_y = prepro.get_LSTM_dataset(train_data, n_timestep, time_interval, input_size, future_day)
    test_x, test_y = prepro.get_LSTM_dataset(test_data, n_timestep, time_interval, input_size, future_day)

    # the firs training dataset
    train_x = train_x[:train_end_back]
    train_y = train_y[:train_end_back]    
    
    # train, validation set 분리하여 train1은 예측모델로 train2는 손절값 학습 모델로 사용
    train1_x, train2_x, train1_y, train2_y = train_test_split(train_x, train_y, test_size=0.5)

    updown = np.sign(test_y[:, -1, 0]).reshape((-1))    

    # model1 training
    early_stopping1 = tf.keras.callbacks.EarlyStopping(patience=2, verbose=1)
    model1.load_weights(checkpoint_path)
    model1.fit(train_x, train_y, batch_size=batch_size, epochs=3, callbacks=[early_stopping1], validation_data=(test_x, test_y))
    model1.save_weights(checkpoint_path)
    
    # model2 training
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)    
    early_stopping2 = learn.EarlyStopping(patience=2, verbose=1)
    iter = epochs
    basic_epochs = tf.cast(epochs / 5, dtype=tf.int32)
    for iteration in range(iter):
        batch_input, batch_output = learn.next_random_interval_batch(train_x, train_y, batch_size, future_day)
        
        gradients2 = gradient2(model1, model2, batch_input, batch_output)
        optimizer.apply_gradients(zip(gradients2, model2.trainable_variables))
        if iteration % 100 == 0:
            rates = test_y / 100
            preds = tf.cast(tf.math.sign(model1(test_x)), dtype=tf.float64)
            profits = tf.cast(1 + tf.convert_to_tensor(rates, dtype=tf.float64)*preds, dtype=tf.float64)

            # model2의 input 생성
            test_x_m2 = tf.concat([preds, rates], 2)

            # model2의 target 생성
            targets = np.zeros(len(profits))
            targets[:-1] = tf.reshape(profits[1:, -1, 0], [-1]) 

            MSE = keras.losses.MSE(tf.reshape(model2(test_x_m2)[:, -1, 0], [-1]), targets)          
           
            print('prediction_MSE =', MSE)

        if iteration > iter / 2 and early_stopping2.validate(MSE)==True:
            break    
    
    if iter > basic_epochs: iter -= basic_epochs
    if iter < basic_epochs: iter = basic_epochs

    print('test dates ' + current_test_start + "~" + current_test_end)
    
    # prediction1 accuracy
    prediction1 = model1.predict(test_x)[:, -1, 0].reshape(-1)
    temp = tf.math.multiply(updown, prediction1)
    accu = tf.reduce_sum(list(map(lambda x: 1 if x > 0 else 0, temp)))/len(temp)
    print('prediction1 accuracy = ', accu)

    # reinforced prediction2
    prediction2 = test(model1, model2, test_x, test_y)    
    temp = tf.math.multiply(updown, prediction2)
    accu = tf.reduce_sum(list(map(lambda x: 1 if x > 0 else 0, temp)))/len(temp)
    print('prediction2 accuracy = ', accu)
    
    test_prediction1.append(prediction1)
    test_prediction2.append(prediction2)
    
    # escape from while
    if current_test_end == test_end:
     break

    #train, start dates shift
    current_train_end = df.loc[prepro.date_to_index(df, current_train_end) + trans_day, 'date']
    current_train_start = df.loc[prepro.date_to_index(df, current_train_end) - 500, 'date']
    current_test_start = df.loc[prepro.date_to_index(df, current_test_start) + trans_day, 'date']
    if prepro.date_to_index(df, test_end) - prepro.date_to_index(df, current_test_start) < trans_day:
        current_test_end = test_end
    else:
        current_test_end = df.loc[prepro.date_to_index(df, current_test_end) + trans_day, 'date']

Train on 4640 samples, validate on 1 samples
Epoch 1/3
4640/4640 [==============================] - 18s 4ms/sample - loss: 26.8941 - val_loss: 2.9142
Epoch 2/3
4640/4640 [==============================] - 10s 2ms/sample - loss: 1.8257 - val_loss: 1.8611
Epoch 3/3
4640/4640 [==============================] - 10s 2ms/sample - loss: 1.0483 - val_loss: 1.3974

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are portin

prediction_MSE = tf.Tensor(0.830423047871264, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8074033028790382, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.7643466121251983, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8018526851447518, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.7570035164937963, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8020743081033288, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8164860254196924, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8449588893578017, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8273613521507741, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.845136526479692, shape=(), dtype=float64)
step :  1
step :  2
step :  3
Training process is stopped early....
test dates 2019-01-08~2019-01-08
prediction1 accuracy =  tf.Tensor(1.0, shape=(), dtype=float64)
prediction2 accuracy =  tf.Tensor(1.0, shape=(), dtype=float64)
Train on 469 samples, validate on 1 samples
Epoch 1/3

prediction2 accuracy =  tf.Tensor(0.0, shape=(), dtype=float64)
Train on 469 samples, validate on 1 samples
Epoch 1/3
469/469 [==============================] - 1s 2ms/sample - loss: -0.0893 - val_loss: -0.3681
Epoch 2/3
469/469 [==============================] - 1s 2ms/sample - loss: -0.0930 - val_loss: -0.3932
Epoch 3/3
469/469 [==============================] - 1s 2ms/sample - loss: -0.0988 - val_loss: -0.3818
prediction_MSE = tf.Tensor(0.8167526468973882, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.7508742429163853, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.7975678188959279, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8141890688872913, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.7767667569141992, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.7977510502210841, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8203884188737085, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.823372077546626, shape=(), dtype=float64)
predicti

469/469 [==============================] - 1s 2ms/sample - loss: -0.0991 - val_loss: 0.1657
Epoch 00003: early stopping
prediction_MSE = tf.Tensor(0.832429070771358, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8345178799608135, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8352047447047539, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.846497532411604, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8440471082767687, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.846226647289555, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8376392539677369, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.839213920149632, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8424545631999223, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.82825876224355, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8260983841183247, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8228780295980052, shape=(), dtype=float64)
prediction_MSE = tf.Te

prediction_MSE = tf.Tensor(0.8385377437444994, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8220043994270441, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8302242622835934, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8207092414816088, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8207877559254975, shape=(), dtype=float64)
step :  1
step :  2
step :  3
Training process is stopped early....
test dates 2019-01-30~2019-01-30
prediction1 accuracy =  tf.Tensor(1.0, shape=(), dtype=float64)
prediction2 accuracy =  tf.Tensor(0.0, shape=(), dtype=float64)
Train on 469 samples, validate on 1 samples
Epoch 1/3
469/469 [==============================] - 1s 2ms/sample - loss: -0.1258 - val_loss: -0.1999
Epoch 2/3
469/469 [==============================] - 1s 2ms/sample - loss: -0.1116 - val_loss: -0.1840
Epoch 3/3
469/469 [==============================] - 1s 2ms/sample - loss: -0.1054 - val_loss: -0.2099
prediction_MSE = tf.Tensor(0.8193696751512825, shape=(), d

...

Epoch 00003: early stopping
prediction_MSE = tf.Tensor(0.8232168604344139, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8148330828761097, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.815522457651241, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8199673736611466, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8220295823663015, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8165942846499128, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8225520261982098, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8180108252863114, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8208979200729551, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8217016939413249, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8235284992767653, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.819071986490826, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.821538854754408, shape=(), dtype=float64)
step :  1
step :  2
step :  3
Training 

prediction_MSE = tf.Tensor(0.8327229760543595, shape=(), dtype=float64)
step :  1
step :  2
step :  3
Training process is stopped early....
test dates 2019-08-27~2019-08-27
prediction1 accuracy =  tf.Tensor(1.0, shape=(), dtype=float64)
prediction2 accuracy =  tf.Tensor(1.0, shape=(), dtype=float64)
Train on 469 samples, validate on 1 samples
Epoch 1/3
469/469 [==============================] - 1s 2ms/sample - loss: -0.1614 - val_loss: -0.0930
Epoch 2/3
469/469 [==============================] - 1s 2ms/sample - loss: -0.1674 - val_loss: 0.6049
Epoch 3/3
469/469 [==============================] - 1s 2ms/sample - loss: -0.1682 - val_loss: -0.1778
prediction_MSE = tf.Tensor(0.8272197459380237, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8212845251026835, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8279686831085655, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8266736021359975, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8291056412202948, shape=(), dt

prediction_MSE = tf.Tensor(0.8305208200040539, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8310081369366209, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8326679328493789, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8344064789677965, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8280860538116173, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8306336996569463, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8277042498502993, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8292652119621948, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8356301194203866, shape=(), dtype=float64)
step :  1
step :  2
step :  3
Training process is stopped early....
test dates 2019-09-04~2019-09-04
prediction1 accuracy =  tf.Tensor(0.0, shape=(), dtype=float64)
prediction2 accuracy =  tf.Tensor(0.0, shape=(), dtype=float64)
Train on 469 samples, validate on 1 samples
Epoch 1/3
469/469 [==============================] - 1s 2ms/sample - loss: -0.1

prediction2 accuracy =  tf.Tensor(0.0, shape=(), dtype=float64)
Train on 469 samples, validate on 1 samples
Epoch 1/3
469/469 [==============================] - 1s 2ms/sample - loss: -0.1718 - val_loss: -0.0896
Epoch 2/3
469/469 [==============================] - 1s 2ms/sample - loss: -0.0063 - val_loss: 0.1095
Epoch 3/3
469/469 [==============================] - 1s 2ms/sample - loss: -0.1342 - val_loss: -0.1784
prediction_MSE = tf.Tensor(0.8304106638117901, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8332659996831495, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8330525118577157, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.836876139893107, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8366793094090532, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8315670159619835, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8283075838463105, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8260565618297875, shape=(), dtype=float64)
predictio

prediction_MSE = tf.Tensor(0.8177496041203227, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8215274015106075, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8133408137965183, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8210424407222696, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.809710948033171, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.810255216194836, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8109241879141251, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.808746884914413, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8020372620572722, shape=(), dtype=float64)
prediction_MSE = tf.Tensor(0.8067728326403234, shape=(), dtype=float64)
step :  1
step :  2
step :  3
Training process is stopped early....
test dates 2019-09-23~2019-09-23
prediction1 accuracy =  tf.Tensor(1.0, shape=(), dtype=float64)
prediction2 accuracy =  tf.Tensor(0.0, shape=(), dtype=float64)
Train on 469 samples, validate on 1 samples
Epoch 1/3


In [ ]:
t1 = np.concatenate(test_prediction1)
t2 = np.concatenate(test_prediction2)

In [ ]:
#t2

In [ ]:
#test_prediction = np.concatenate(test_prediction, axis=0).reshape((-1, n_timestep, 1))
#train_prediction = learn.predict_batch_test(model1, train_x[:batch_size], len(train_x[:batch_size]))

# 원 지수, 날짜는 target conversion이 되기 전 dataframe에서 가져옴
test_dates, test_base_prices, train_dates, train_base_prices = prepro.get_test_dates_prices(dataframe, test_start, test_end,
                                                      train_start, train_end, n_timestep, time_interval, future_day, target_column)

# 전체 test_oouput 생성
_, test_data = prepro.get_train_test_data(df, target_column, remove_columns,
                                                   train_start, train_end,
                                                   test_start, test_end,
                                                   future_day, n_timestep, time_interval)
_, test_y = prepro.get_LSTM_dataset(test_data, n_timestep, time_interval, input_size, future_day)

In [ ]:
updown = np.sign(test_y[:, -1, 0]).reshape((-1))
#calculate accuracy
temp = tf.math.multiply(updown, t1.reshape((-1)))
accu = tf.reduce_sum(list(map(lambda x: 1 if x > 0 else 0, temp)))/len(temp)
print('accuracy = ', accu)

In [ ]:
updown.shape

In [ ]:
#train_base_prices = train_base_prices[:batch_size]
#train_prediction = train_prediction[:batch_size]
#train_y = train_y[:batch_size]

In [ ]:
result = GenerateResult(t1, t2, test_y[:, -1, 0].reshape(-1), test_dates, n_timestep, future_day, trans_day)

In [ ]:
#result.extract_last_output()
result.convert_price(test_base_prices,conversion_type=target_type)

In [ ]:
result.evaluation()
result.table()
result.save_result(model_name,item_name,n_unit,target_type,batch_size,n_timestep,time_interval,epochs,str(alpha),comment)
result.save_visualization()
result.save_model(model2)

In [ ]:
"""
current_train_start = df.loc[prepro.date_to_index(df, train_start), 'date']
current_train_end = df.loc[prepro.date_to_index(df, train_end), 'date']
current_test_start = df.loc[prepro.date_to_index(df, test_start), 'date']
current_test_end = df.loc[prepro.date_to_index(df, test_start) + trans_day - 1, 'date']


#  각 transfer 구간의 예측값들을 합치기 위하여
test_prediction1 = []
test_prediction2 = []
test_target = []

# early_stopping = tf.keras.callbacks.EarlyStopping(patience=3, verbose=1)
early_stopping = learn.EarlyStopping(patience=2, verbose=1)


gc.collect()

train_data, test_data = prepro.get_train_test_data(df, target_column, remove_columns, 
                                                       current_train_start, current_train_end,
                                                       current_test_start, current_test_end,
                                                       future_day, n_timestep, time_interval)

# input_size, columns reset
input_size = len(df.columns) - len(remove_columns)
input_columns = df.columns.copy()

train_x, train_y = prepro.get_LSTM_dataset(train_data, n_timestep, time_interval, input_size, future_day)
test_x, test_y = prepro.get_LSTM_dataset(test_data, n_timestep, time_interval, input_size, future_day)

#model.compile(optimizer='adam',
#          loss=loss_fn)
#          #callbacks=[cp-callback]
#          #metrics=['accuracy'])

# the firs training dataset
train_x = train_x[:-future_day]
train_y = train_y[:-future_day]    

#global_step = tf.train.get_or_create_global_step()
global_step = tf.Variable(0, trainable=False)
#lr_decay = tf.train.exponential_decay(learning_rate, global_step,
#                                      train_input.shape[0]/batch_size*5, 0.5, staircase=True)
lr_decay = tf.compat.v1.train.exponential_decay(learning_rate,global_step, int(len(train_y)/batch_size), 0.96, staircase=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
#optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

updown = np.sign(test_y[:, -1, 0]).reshape((-1))    
epochs = len(train_y)
for iteration in range(399):
    batch_input, batch_output = learn.next_random_interval_batch(train_x, train_y, batch_size, future_day)

    #noise = 2*np.random.randn(batch_size,n_timestep,1)
    #batch_output = batch_output+noise
    #batch_input = encoder(train_input[idx])
    gradients1 = gradient1(model1, model2, batch_input, batch_output)
    optimizer.apply_gradients(zip(gradients1, model1.trainable_variables))
    
    targets = tf.reshape(train_y[:, -1, 0], [-1])
    rates = targets / 100
    preds = tf.reshape(model1(train_x, training=False)[:, -1, 0], [-1])
    
    n = len(targets)    
    returns = [1.0]
    losses = []
    for i in range(n - 1):
       
        # average_return, std of returns, remaining days, preds[0] 
        state = []
        
        random_rates = []
        for k in range(i+1):
            random_rates.append(rates[k])
        # 실재 수익률을 기반으로 random 수익률 생성, 예측에 의한 porfits 생성 
        for k in range(i+1, n):
            random_rates.append(tf.random.normal((), mean=targets[k] / 100, stddev=0.1, dtype=tf.float64))
        profits = tf.convert_to_tensor(random_rates, dtype=tf.float64)*tf.cast(preds, dtype=tf.float64) + 1
        
        # 현재까지의 예측에 의한 수익률 기하평균 구하기
        returns_past = []
        for k in range(i+1):
            returns_past.append(profits[k])
        avg_return = tf.cast(tf.math.reduce_prod(returns_past)**(1/(i+1)), dtype=tf.float64)
        
        state.append(avg_return)
        state.append(tf.math.reduce_std(returns))
        state.append((n - i) / n)
        state.append(preds[i])
        state = np.array(state).reshape((1, 4))
     
        # 목표일까지의 기대 기하 평균 수익률 구하기 
        returns_future = []
        for j in range(i+1, n):
            returns_future.append(profits[j])
        avg_return_future = tf.math.reduce_prod(returns_future)**(1/(n-i-1))

        # 예측 기하 평균 수익률과 기대 기하 평균 수익률의 MSE
        value = tf.cast(model2(state, training=True)[0, 0], dtype=tf.float64)
        losses.append((value - avg_return_future)**2)
        if n == 3: break
    print("losses", losses)
    print("value", value)    
    with tf.GradientTape() as tape:
        gradients2 = tape.gradient(tf.math.reduce_sum(losses), model2.trainable_variables)
    optimizer.apply_gradients(zip(gradients2, model2.trainable_variables))
    

    
    if iteration % 100 == 0:
        #test_MSE = model.evaluate(test_x, test_y)
        prediction = model1.predict(test_x)
        prediction_MSE = sum((updown - np.sign(prediction[:, -1, 0]).reshape(-1))**2)/len(test_y)
        print('prediction_MSE =', prediction_MSE)

    if iteration > epochs / 2 and early_stopping.validate(prediction_MSE)==True:
        break
epochs -= epochs / 5
if epochs <= 0: epochs = 100
"""